In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset/train.csv", index_col=0)
df_bk = df
df.sample(5)

,label,tweet
id,,
8179,0,"""flcrashcorrectors: ""kscrashcorrectors: ""kscra..."
15427,0,this is where it happened ð #pray #disney ...
888,0,#decors buffalo simulation: buffalo for you ...
16002,1,"#sikh #temple vandalised in in #calgary, #wso ..."
7315,0,@user just waiting for john mccain to announc...


Twits should have less than 140ch

In [3]:
df[df['tweet'].str.len() > 140].head()

,label,tweet
id,,
8,0,the next school year is the year for exams.ð...
336,0,watching the @user leadership embrace &amp; ki...
434,0,@user * sweets ð°ð©ð¬ð­ð«â¨!!!!!! ...
475,0,this is a shockingly and increasingly #viole...
566,0,thank youðsoo muchð pretty @user i feel ...


In [4]:
df[df['tweet'].str.len() > 140].shape

(222, 2)

There must be some f*k simbols encoded in here

In [5]:
df.tweet[336]

"watching the @user leadership embrace &amp; kiss donald's ass is #pathetic &amp;  . this is now the @user pay. a pay of racism &amp; hate. #ripgop"

In [6]:
from bs4 import BeautifulSoup
BeautifulSoup(df.tweet[336], 'lxml').get_text()

"watching the @user leadership embrace & kiss donald's ass is #pathetic &  . this is now the @user pay. a pay of racism & hate. #ripgop"

In [7]:
df['tweet'] = df['tweet'].apply(lambda x : BeautifulSoup(x, 'lxml').get_text())

In [8]:
df[df['tweet'].str.len() > 140].shape

(149, 2)

In [9]:
df.tweet[434]

'@user * sweets ð\x9f\x8d°ð\x9f\x8d©ð\x9f\x8d¬ð\x9f\x8d\xadð\x9f\x8d«â\x9c¨!!!!!! * #instagood #ì\x95\x84ì\x9d´ë\x8f\x8c #å\x81¶å\x83\x8f #idola #idol #à¹\x84à¸\xadà¸\x94à¸\xadà¸¥ #followme #selfie   #ã\x81©ã\x82\x8bã\x81\x88ã\x82\x8câ\x80¦ '

In [10]:
import re
df['tweet'] = df['tweet'].apply(lambda x : re.sub(r'@[A-Za-z0-9]+','',x))

In [11]:
df.tweet[434]

' * sweets ð\x9f\x8d°ð\x9f\x8d©ð\x9f\x8d¬ð\x9f\x8d\xadð\x9f\x8d«â\x9c¨!!!!!! * #instagood #ì\x95\x84ì\x9d´ë\x8f\x8c #å\x81¶å\x83\x8f #idola #idol #à¹\x84à¸\xadà¸\x94à¸\xadà¸¥ #followme #selfie   #ã\x81©ã\x82\x8bã\x81\x88ã\x82\x8câ\x80¦ '

In [12]:
df.shape

(31962, 2)

In [13]:
df.tweet.str.contains('https|http', na=False).count()

31962

Cleaning special simbols and numbers like # and @ -> better f-1 without this

In [14]:
df.tweet[1]

' when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [15]:
#df['tweet'] = df['tweet'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

In [16]:
df.tweet[1]

' when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

Lemmatization

In [17]:
import nltk

In [18]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/rcrespillo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
def tweets_lemm(tweet):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(tweet)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [20]:
print(tweets_lemm(df.tweet[1]))

when a father is dysfunctional and is so selfish he drag his kid into his dysfunction . # run


In [21]:
df['tweet'] = df['tweet'].apply(lambda x : tweets_lemm(x))

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X = df["tweet"]
y = df["label"]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify=y)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [26]:
nltk.download('punkt')
nltk.download('stopwords')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /home/rcrespillo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rcrespillo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
vectorizer = TfidfVectorizer(analyzer='word', 
                             tokenizer=nltk.word_tokenize, 
                             stop_words=stops, 
                             ngram_range=(1,3))

X_train_v = vectorizer.fit_transform(X_train)
X_test_v = vectorizer.transform(X_test)

In [28]:
X_train_v.shape

(25569, 377046)

In [29]:
vectorizer.vocabulary_

{'going': 184480,
 'seeing': 299149,
 'everyday': 159618,
 'look': 228180,
 'phone': 270972,
 'ð\x9f\x98\x94': 375011,
 '#': 6111,
 'verysad': 344591,
 'going seeing': 184936,
 'seeing everyday': 299195,
 'everyday look': 159661,
 'look phone': 228491,
 'phone ð\x9f\x98\x94': 271051,
 'ð\x9f\x98\x94 #': 375012,
 '# verysad': 45393,
 'going seeing everyday': 184937,
 'seeing everyday look': 299196,
 'everyday look phone': 159662,
 'look phone ð\x9f\x98\x94': 228492,
 'phone ð\x9f\x98\x94 #': 271052,
 'ð\x9f\x98\x94 # verysad': 375028,
 "n't": 248553,
 'need': 252150,
 'blame': 113979,
 'atrocity': 101748,
 'isi': 209917,
 'islamicterorrist': 210057,
 '-': 64583,
 'nut': 259265,
 'everywhere': 160212,
 'commit': 132482,
 'terrorism': 325207,
 '.': 66124,
 'lgbtresilience': 222273,
 "n't need": 249655,
 'need blame': 252242,
 'blame atrocity': 113990,
 'atrocity #': 101749,
 '# isi': 25622,
 'isi #': 209918,
 '# islamicterorrist': 25651,
 'islamicterorrist -': 210058,
 '- nut': 65404,
 'n

In [30]:
from sklearn.neural_network import MLPClassifier

In [31]:
mlp_clf = MLPClassifier(activation='tanh',  
                     hidden_layer_sizes=(10,), 
                     verbose=True)

In [32]:
mlp_clf.fit(X_train_v, y_train)

Iteration 1, loss = 0.38205469
Iteration 2, loss = 0.19653906
Iteration 3, loss = 0.13642463
Iteration 4, loss = 0.09583283
Iteration 5, loss = 0.06509543
Iteration 6, loss = 0.04443694
Iteration 7, loss = 0.03149561
Iteration 8, loss = 0.02335673
Iteration 9, loss = 0.01811265
Iteration 10, loss = 0.01459987
Iteration 11, loss = 0.01216037
Iteration 12, loss = 0.01040312
Iteration 13, loss = 0.00910236
Iteration 14, loss = 0.00810776
Iteration 15, loss = 0.00732779
Iteration 16, loss = 0.00670485
Iteration 17, loss = 0.00619337
Iteration 18, loss = 0.00576763
Iteration 19, loss = 0.00540630
Iteration 20, loss = 0.00509656
Iteration 21, loss = 0.00482422
Iteration 22, loss = 0.00458320
Iteration 23, loss = 0.00436866
Iteration 24, loss = 0.00417446
Iteration 25, loss = 0.00399749
Iteration 26, loss = 0.00383628
Iteration 27, loss = 0.00368654
Iteration 28, loss = 0.00354670
Iteration 29, loss = 0.00341733
Iteration 30, loss = 0.00329543
Iteration 31, loss = 0.00317684
Iteration 32, los

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [33]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report

In [34]:
y_pred = mlp_clf.predict(X_test_v)

In [35]:
#confusion_matrix(y_test, y_pred)

In [36]:
print("roc_auc score: {}".format(roc_auc_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

roc_auc score: 0.7993207827706356
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      5945
          1       0.92      0.60      0.73       448

avg / total       0.97      0.97      0.97      6393



In [37]:
X_all = vectorizer.transform(X)
y_all = y

In [38]:
mlp_clf.fit(X_all, y_all)

Iteration 1, loss = 0.39734188
Iteration 2, loss = 0.19497688
Iteration 3, loss = 0.13580592
Iteration 4, loss = 0.09539689
Iteration 5, loss = 0.06516864
Iteration 6, loss = 0.04535579
Iteration 7, loss = 0.03296804
Iteration 8, loss = 0.02508637
Iteration 9, loss = 0.01986712
Iteration 10, loss = 0.01625697
Iteration 11, loss = 0.01367538
Iteration 12, loss = 0.01175884
Iteration 13, loss = 0.01030564
Iteration 14, loss = 0.00917269
Iteration 15, loss = 0.00827120
Iteration 16, loss = 0.00753405
Iteration 17, loss = 0.00692051
Iteration 18, loss = 0.00640447
Iteration 19, loss = 0.00596429
Iteration 20, loss = 0.00558073
Iteration 21, loss = 0.00524437
Iteration 22, loss = 0.00494511
Iteration 23, loss = 0.00467603
Iteration 24, loss = 0.00443348
Iteration 25, loss = 0.00421292
Iteration 26, loss = 0.00400989
Iteration 27, loss = 0.00382425
Iteration 28, loss = 0.00365250
Iteration 29, loss = 0.00349274
Iteration 30, loss = 0.00334365
Iteration 31, loss = 0.00320370
Iteration 32, los

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [39]:
df_test = pd.read_csv("dataset/test.csv")

In [40]:
df_test.sample(5)

,id,tweet
13662,45625,@user @user &amp; will i am... ð¤ðð¼ð...
1149,33112,@user ð¦ðwhen haagen-dazs comes to brigh...
8939,40902,"@user didn't know there was one, just looked i..."
1349,33312,what america owes #blacks #africanamerican ...
15048,47011,to see my fellow #ham radio operators #cantw...


In [41]:
dtest = vectorizer.transform(df_test['tweet'])
test_pred = mlp_clf.predict(dtest)

In [42]:
df_test['label'] = test_pred

In [43]:
submission = df_test[['id','label']]

In [44]:
submission.to_csv('sub_rodixxi_mlpc_lemmatizer.csv', index=False)